In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta
from geographiclib.geodesic import Geodesic as GD
from scipy.interpolate import interpn
import folium
from datetime import datetime, timedelta
import pytz
import torch
import torch.nn as nn
np.set_printoptions(precision=4, suppress=True)

from minio import Minio
import tempfile

In [3]:
import config as cfg

# WARNING

This notebook is was not fully updated to the code in `preprocessing.py` and `run_model.py`! It was updated to be executable but some differences may exist.

Remaining issue:
1. weather forecasts are from NOAA. will there be timezone difference between weather forecast and the input timestamp?
2. The current timestamp and location is given as examples. Will it be provided in json?
3. There is a chance that the given waypoints's latitude is outside the experimenting area.
4. If the arriving time of the points in the experimenting area is larger than 384 hours given the weather forecast?

## steps
1. read in files (json or csv) and parse the data into dataframe
2. check if timestamp is in accending order
3. longitude convertion: western negative to 0-360 format
4. according to the current position and timestamp, filter out the needed waypoints to df_filter
5. create section_waypoints
6. generate interpolated coordinates, timestamps and write cog
7. interpolate corresponding weather attributes for all the spatial points in the experimenting area
8. do the transformation for weather attributes (difference between previous forecasted and latest forecasted)
9. zero padding
10. pass through the model

In [4]:
# longitude list and latitude list of the grids in the exaperimenting area
long_list = np.array([156.00000694, 157.25000699, 158.50000705, 159.7500071 ,
                      161.00000716, 162.25000722, 163.50000727, 164.75000733,
                      166.00000738, 167.25000744, 168.50000749, 169.75000755,
                      171.00000761, 172.25000766, 173.50000772, 174.75000777,
                      176.00000783, 177.25000788, 178.50000794, 179.75000799,
                      181.00000805, 182.25000811, 183.50000816, 184.75000822,
                      186.00000827, 187.25000833, 188.50000838, 189.75000844,
                      191.00000849, 192.25000855, 193.50000861, 194.75000866,
                      196.00000872, 197.25000877, 198.50000883, 199.75000888,
                      201.00000894, 202.250009  , 203.50000905, 204.75000911,
                      206.00000916, 207.25000922, 208.50000927, 209.75000933,
                      211.00000938, 212.25000944, 213.5000095 , 214.75000955,
                      216.00000961, 217.25000966, 218.50000972, 219.75000977,
                      221.00000983, 222.25000988, 223.50000994, 224.75001   ,
                      226.00001005, 227.25001011, 228.50001016])
lat_list = np.array([50.  , 49.75, 49.5 , 49.25, 49.  , 48.75, 48.5 , 48.25, 48.  ,
       47.75, 47.5 , 47.25, 47.  , 46.75, 46.5 , 46.25, 46.  , 45.75,
       45.5 , 45.25, 45.  , 44.75, 44.5 , 44.25, 44.  , 43.75, 43.5 ,
       43.25, 43.  , 42.75, 42.5 , 42.25, 42.  , 41.75, 41.5 , 41.25,
       41.  , 40.75, 40.5 , 40.25, 40.  , 39.75, 39.5 , 39.25, 39.  ,
       38.75, 38.5 , 38.25, 38.  , 37.75, 37.5 , 37.25, 37.  , 36.75,
       36.5 , 36.25, 36.  , 35.75, 35.5 , 35.25, 35.  , 34.75, 34.5 ,
       34.25, 34.  , 33.75, 33.5 , 33.25, 33.  , 32.75, 32.5 , 32.25,
       32.  , 31.75, 31.5 , 31.25, 31.  , 30.75, 30.5 , 30.25, 30.  ,
       29.75, 29.5 , 29.25, 29.  , 28.75, 28.5 , 28.25, 28.  , 27.75,
       27.5 , 27.25, 27.  , 26.75, 26.5 , 26.25, 26.  , 25.75, 25.5 ,
       25.25])
print(long_list.shape)
print(lat_list.shape)

range_ul = [50, 156.00000694]  # upper left corner coordinates of the experimenting area
range_lr = [25, 228.50001016]  # lower right corner coordinates of the experimenting area

(59,)
(100,)


### 1. load input data

In [5]:
# Option 1: load the JSON file
# the provided example is not in the northen pacific area
with open('input_sample.json', 'r') as f:
    data = json.load(f)

# create a dictionary to store the data
parsed_data = {'latitude': [], 'longitude': [], 'timestamp': []}

# iterate over each coordinate and timestamp, and store them in the dictionary
for i in range(len(data['coordinates'])):
    parsed_data['latitude'].append(data['coordinates'][i]['latitude'])
    parsed_data['longitude'].append(data['coordinates'][i]['longitude'])
    parsed_data['timestamp'].append(data['timestamps'][i])

# create a Pandas DataFrame from the parsed data
input_df = pd.DataFrame(parsed_data)

print(input_df)

    latitude  longitude             timestamp
0  60.153317  24.948967  2018-10-30T00:00:00Z
1  58.549169  21.042663  2018-10-30T11:18:46Z
2  56.468645  17.524141  2018-10-30T23:44:34Z
3  55.792767  15.700957  2018-10-31T05:08:32Z
4  54.651725  12.375373  2018-10-31T14:56:09Z
5  53.966667  10.900000  2018-10-31T19:48:06Z


In [6]:
# Option 2: read the csv file into a DataFrame
input_df = pd.read_csv('98765434_voyage-export.csv')

# extract the desired variables
variables = ['LATITUDE', 'LONGITUDE', 'TIME AT WAYPOINT']
input_df = input_df[variables]
input_df = input_df.rename(columns={'TIME AT WAYPOINT': 'timestamp', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})


In [7]:
input_df

,latitude,longitude,timestamp
0,31.100832,123.034870,2022-10-09 16:00:00 +00:00
1,31.114708,123.064547,2022-10-09 16:05:13 +00:00
2,32.094767,125.222639,2022-10-09 22:20:45 +00:00
3,32.227666,125.531273,2022-10-09 23:13:36 +00:00
4,32.988736,126.654146,2022-10-10 02:52:20 +00:00
5,33.140617,127.008370,2022-10-10 03:52:28 +00:00
6,34.370772,128.934025,2022-10-10 09:56:17 +00:00
7,34.657018,129.087981,2022-10-10 10:52:35 +00:00
8,35.092269,129.488136,2022-10-10 12:30:43 +00:00
9,35.351208,129.833290,2022-10-10 13:39:41 +00:00


### 2 + 3 timestamp and longitude conversion 

In [8]:
# convert datatime
input_df['timestamp'] = pd.to_datetime(input_df['timestamp'], format='ISO8601')
input_df = input_df.sort_values('timestamp')

# convert longitudes in the western hemisphere to positive numbers
input_df.loc[input_df['longitude'] < 0, 'longitude'] += 360

input_df = gpd.GeoDataFrame(input_df, geometry=gpd.points_from_xy(input_df.longitude, input_df.latitude), crs=4326)

In [9]:
input_df.dtypes

latitude                 float64
longitude                float64
timestamp    datetime64[ns, UTC]
geometry                geometry
dtype: object

In [10]:
print(input_df.shape)
input_df.head()

(43, 4)


,latitude,longitude,timestamp,geometry
0,31.100832,123.034870,2022-10-09 16:00:00+00:00,POINT (123.03487 31.10083)
1,31.114708,123.064547,2022-10-09 16:05:13+00:00,POINT (123.06455 31.11471)
2,32.094767,125.222639,2022-10-09 22:20:45+00:00,POINT (125.22264 32.09477)
3,32.227666,125.531273,2022-10-09 23:13:36+00:00,POINT (125.53127 32.22767)
4,32.988736,126.654146,2022-10-10 02:52:20+00:00,POINT (126.65415 32.98874)


### 4. filter needed waypoints according to current T0 and current position G0 (lat0, lon0) 

In [11]:
#current T0 and current position G0 (lat0, lon0) should be provided by json file
# Now define T0 and G0 as examples

# example 1: starting point is where the ship not yet entered the experimenting area
#T0_str = '2022-10-09 17:05:13+00:00'
#G0 = (32,124)
#T0 = pd.Timestamp(T0_str, tz='UTC')

# example 2: starting point is where the ship is in the middle of the experimenting area
T0_str = '2022-10-14 16:28:15+00:00'
G0 = (49,171)
T0 = pd.Timestamp(T0_str, tz='UTC')

# example 3: starting point is where the ship has already sailed outside the experimenting area
#T0_str = '2022-10-21 07:07:33+00:00'
#G0 = (34.313492,239.532944)
#T0 = pd.Timestamp(T0_str, tz='UTC')
           
# filter the dataframe by timestamp and longitude
mask = (input_df['timestamp'] >= T0) & (input_df['longitude'].between(range_ul[1], range_lr[1]))

# check if the current point or timestamp is after/ exceed the experimenting area
if mask.any():
    # do the filtering
    index_first = np.nonzero(mask.values)[0][0]
    index_last = np.nonzero(mask.values)[0][-1]
    df_filtered = input_df[index_first-1:index_last+2]
else:
    # Report
    print('Error: rerouting point outside the right bound of the experimenting area')
    # also need to stop excuting at all



In [12]:
'''in case of the 3nd senario, quit from here'''

'in case of the 3nd senario, quit from here'

In [13]:
df_filtered

,latitude,longitude,timestamp,geometry
21,47.429808,170.247897,2022-10-14 15:28:15+00:00,POINT (170.24790 47.42981)
22,47.592222,183.484019,2022-10-15 18:21:38+00:00,POINT (183.48402 47.59222)
23,46.300341,190.204689,2022-10-16 08:42:10+00:00,POINT (190.20469 46.30034)
24,44.186511,201.717278,2022-10-17 09:53:27+00:00,POINT (201.71728 44.18651)
25,43.639264,202.444757,2022-10-17 12:10:04+00:00,POINT (202.44476 43.63926)
26,43.000798,206.117764,2022-10-17 20:26:08+00:00,POINT (206.11776 43.00080)
27,41.992089,212.587190,2022-10-18 11:06:16+00:00,POINT (212.58719 41.99209)
28,40.674197,223.251860,2022-10-19 11:30:41+00:00,POINT (223.25186 40.67420)
29,39.550858,228.689809,2022-10-20 00:28:14+00:00,POINT (228.68981 39.55086)


### 5. waypoints_section

In [14]:
# store the information between each pair of consecutive waypoints from df_filtered in sections_waypoints

# calculating bearing
def get_bearing(lat1, lat2, long1, long2):
    '''The function is used to calculate the CoG from node 1 to node 2'''
    brng = GD.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    if brng < 0:
        brng = brng + 360
    return brng


sections_waypoints = pd.DataFrame(columns=['E_latitude', 'E_longitude', 'L_latitude', 
                                                'L_longitude', 'E_timestamp', 'L_timestamp', 'cog'])

for i in range(len(df_filtered) - 1):
    row1 = df_filtered.iloc[i]
    row2 = df_filtered.iloc[i + 1]

    # Calculate course over ground between two waypoints
    cog = get_bearing(row1['latitude'], row2['latitude'], row1['longitude'], row2['longitude'])

    # Add the information to the sections_waypoints dataframe
    sections_waypoints = pd.concat([sections_waypoints, pd.Series({
            'E_latitude': row1['latitude'],
            'E_longitude': row1['longitude'],
            'E_timestamp': row1['timestamp'],
            'L_latitude': row2['latitude'],
            'L_longitude': row2['longitude'],
            'L_timestamp': row2['timestamp'],
            'cog': cog
    }).to_frame().T], ignore_index=True)
sections_waypoints['E_latitude'] = sections_waypoints['E_latitude'].astype(float)
sections_waypoints['E_longitude'] = sections_waypoints['E_longitude'].astype(float)
sections_waypoints['L_latitude'] = sections_waypoints['L_latitude'].astype(float)
sections_waypoints['L_longitude'] = sections_waypoints['L_longitude'].astype(float)
sections_waypoints['E_timestamp'] = pd.to_datetime(sections_waypoints['E_timestamp'])
sections_waypoints['L_timestamp'] = pd.to_datetime(sections_waypoints['L_timestamp'])
sections_waypoints['cog'] = sections_waypoints['cog'].astype(float)
sections_waypoints

,E_latitude,E_longitude,L_latitude,L_longitude,E_timestamp,L_timestamp,cog
0,47.429808,170.247897,47.592222,183.484019,2022-10-14 15:28:15+00:00,2022-10-15 18:21:38+00:00,84.076909
1,47.592222,183.484019,46.300341,190.204689,2022-10-15 18:21:38+00:00,2022-10-16 08:42:10+00:00,103.204608
2,46.300341,190.204689,44.186511,201.717278,2022-10-16 08:42:10+00:00,2022-10-17 09:53:27+00:00,100.426250
3,44.186511,201.717278,43.639264,202.444757,2022-10-17 09:53:27+00:00,2022-10-17 12:10:04+00:00,135.886520
4,43.639264,202.444757,43.000798,206.117764,2022-10-17 12:10:04+00:00,2022-10-17 20:26:08+00:00,102.126772
5,43.000798,206.117764,41.992089,212.587190,2022-10-17 20:26:08+00:00,2022-10-18 11:06:16+00:00,99.698964
6,41.992089,212.587190,40.674197,223.251860,2022-10-18 11:06:16+00:00,2022-10-19 11:30:41+00:00,95.756516
7,40.674197,223.251860,39.550858,228.689809,2022-10-19 11:30:41+00:00,2022-10-20 00:28:14+00:00,103.295024


### 6. generate interpolated coordinates, timestamp and write cog

In [15]:
# Create an empty pandas DataFrame with column 'longitude'
interpolation_full = pd.DataFrame({'longitude': long_list})

# Add empty columns to the DataFrame
interpolation_full['latitude'] = np.nan    # longitudes are given according to the grids
                                           # latitudes are interpolated
interpolation_full['cog'] = np.nan         
interpolation_full['timestamp'] = pd.NaT
interpolation_full['arriving_time_latest'] = np.nan
interpolation_full['arriving_time_previous'] = np.nan

interpolation_full.head()

,longitude,latitude,cog,timestamp,arriving_time_latest,arriving_time_previous
0,156.000007,NaN,NaN,NaT,NaN,NaN
1,157.250007,NaN,NaN,NaT,NaN,NaN
2,158.500007,NaN,NaN,NaT,NaN,NaN
3,159.750007,NaN,NaN,NaT,NaN,NaN
4,161.000007,NaN,NaN,NaT,NaN,NaN


In [16]:
def intp_lat_ts_cog(row):
    """
    Finds the row in `sections_waypoints` where the logitudes in the grid `t_lon` falls between
    the `E_longitude` and `L_longitude` values. Calculates the latitude
    and timestamp for the target point T_point and sets the `latitude`,
    `timestamp`, and `cog` columns in accordingly.

    Parameters:
    t_lon (float): The longitude value to search for in `sections_waypoints`.
    sections_waypoints (pd.DataFrame): The DataFrame containing the sections and waypoints.

    Returns:
    pd.DataFrame: a row with interpolated values of latitude, timestamp and cog
    """


    # Get t_lon value 
    t_lon = row['longitude']

    # Find row in `sections_waypoints` where `t_lon` falls between
    #         the `E_longitude` and `L_longitude` values
    mask = (sections_waypoints['E_longitude'] <= t_lon) & (t_lon <= sections_waypoints['L_longitude'])
    row_sw = sections_waypoints.loc[mask]

    # Calculate t_lat using linear interpolation
    point1 = np.array([row_sw['E_latitude'], row_sw['E_longitude']])
    point2 = np.array([row_sw['L_latitude'], row_sw['L_longitude']])
    t_lat = point1[0] + ((t_lon - point1[1]) / (point2[1] - point1[1])) * (point2[0] - point1[0])

    # Calculate t_timestamp using linear interpolation
    e_ts = row_sw['E_timestamp']
    l_ts = row_sw['L_timestamp']

    t_ts = e_ts + ((t_lon - point1[1])[0] / (point2[1] - point1[1])[0]) * (l_ts - e_ts)

    return pd.Series({'latitude': t_lat[0], 'timestamp': t_ts.values[0], 'cog': row_sw['cog'].iloc[0]})

In [17]:
# return the index of the smallest longitude in grids that is larger than the current position G0[1]
s_grids_lon_index = (interpolation_full['longitude'] >= G0[1]).idxmax()

In [18]:
interpolation_full.loc[s_grids_lon_index:, ['latitude', 'timestamp', 'cog']] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                                .apply(intp_lat_ts_cog, axis=1))
interpolation_full

,longitude,latitude,cog,timestamp,arriving_time_latest,arriving_time_previous
0,156.000007,NaN,NaN,NaT,NaN,NaN
1,157.250007,NaN,NaN,NaT,NaN,NaN
2,158.500007,NaN,NaN,NaT,NaN,NaN
3,159.750007,NaN,NaN,NaT,NaN,NaN
4,161.000007,NaN,NaN,NaT,NaN,NaN
5,162.250007,NaN,NaN,NaT,NaN,NaN
6,163.500007,NaN,NaN,NaT,NaN,NaN
7,164.750007,NaN,NaN,NaT,NaN,NaN
8,166.000007,NaN,NaN,NaT,NaN,NaN
9,167.250007,NaN,NaN,NaT,NaN,NaN


In [19]:
# purple: the current point
# green: inputed waypoints
# red: filtered waypoints according to the experimenting area
# blue: interpolated in order to go through the pretrained model
m = folium.Map(location=[40, 180], zoom_start=3)

# add a smaller marker for each point in the dataframe
 


for i in input_df.index:
    folium.Marker([input_df['latitude'][i], input_df['longitude'][i]], icon=folium.Icon(icon='circle', prefix='fa', color='green')).add_to(m)
for i in interpolation_full.loc[s_grids_lon_index:,:].index:
    folium.Marker([interpolation_full.loc[s_grids_lon_index:,:]['latitude'][i], interpolation_full.loc[s_grids_lon_index:,:]['longitude'][i]], icon=folium.Icon(icon='dot', prefix='fa', color='blue')).add_to(m)#   
for i in df_filtered.index:
    folium.Marker([df_filtered['latitude'][i], df_filtered['longitude'][i]], icon=folium.Icon(icon='heart', prefix='fa', color='red')).add_to(m)#

    
#current point in purple
folium.Marker([G0[0], G0[1]], icon=folium.Icon(icon='circle', prefix='fa', color='purple')).add_to(m)

# create a rectangle with the two corners as bounds
rect = folium.Rectangle(bounds=[range_ul, range_lr], color='green', fill_opacity=0.2)

# add the rectangle to the map
rect.add_to(m)
# display the map
m

### 7. read and interpolate corresponding weather attributes

In [20]:
# find out the lastest available weather forecast according to the current timepoint

def get_last_weather_forecast(timestamp_str):
    
    '''return the forecast files names and the timestamps of the forecast'''
    # only keep the year month day hour minute seconds
    truncated_timestamp_str = timestamp_str[:19]
    
    dt = datetime.strptime(truncated_timestamp_str, '%Y-%m-%d %H:%M:%S')
    forecast_hour = (dt.hour // 6) * 6
    last_forecast = pytz.utc.localize(datetime(dt.year, dt.month, dt.day, forecast_hour))
    previous_forecast = last_forecast-timedelta(hours = 6)
    
    # the most recent forecast and the previous one
    last_forecast_name = (str(last_forecast.year)+str(last_forecast.month).zfill(2)+str(last_forecast.day).zfill(2)
                             +str(last_forecast.hour).zfill(2))
    
    previous_forecast_name = (str(previous_forecast.year)+str(previous_forecast.month).zfill(2)
                              +str(previous_forecast.day).zfill(2)+str(previous_forecast.hour).zfill(2))
    return (last_forecast_name, previous_forecast_name),(last_forecast, previous_forecast) 


In [21]:
forecast_name, forecast_ts = get_last_weather_forecast(T0_str)
print(forecast_name)
print(forecast_ts)

('2022101412', '2022101406')
(datetime.datetime(2022, 10, 14, 12, 0, tzinfo=<UTC>), datetime.datetime(2022, 10, 14, 6, 0, tzinfo=<UTC>))


In [22]:
# Define a function to calculate the time difference in hours 
# between the arriving time at a location and the starting time of the weather forecast
time_diff0 = lambda x: (pytz.utc.localize(x['timestamp']) - forecast_ts[0]) / timedelta(hours=1)
time_diff1 = lambda x: (pytz.utc.localize(x['timestamp']) - forecast_ts[1]) / timedelta(hours=1)

# Apply the function to the dataframe
interpolation_full.loc[s_grids_lon_index:, 'arriving_time_latest'] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(time_diff0, axis=1))
interpolation_full.loc[s_grids_lon_index:, 'arriving_time_previous'] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(time_diff1, axis=1))

In [23]:
interpolation_full

,longitude,latitude,cog,timestamp,arriving_time_latest,arriving_time_previous
0,156.000007,NaN,NaN,NaT,NaN,NaN
1,157.250007,NaN,NaN,NaT,NaN,NaN
2,158.500007,NaN,NaN,NaT,NaN,NaN
3,159.750007,NaN,NaN,NaT,NaN,NaN
4,161.000007,NaN,NaN,NaT,NaN,NaN
5,162.250007,NaN,NaN,NaT,NaN,NaN
6,163.500007,NaN,NaN,NaT,NaN,NaN
7,164.750007,NaN,NaN,NaT,NaN,NaN
8,166.000007,NaN,NaN,NaT,NaN,NaN
9,167.250007,NaN,NaN,NaT,NaN,NaN


In [24]:
def hour2timeframe(row):
    '''The timeframe is from 0 to 209, but the real hour is from 0 to 384. 
    From hour 120 onwards, every three hours per timeframe.
    The function is to find out the positions in the timeframes according to the hour'''
    if row['arriving_time_latest'] > 120:
        tf_latest = (row['arriving_time_latest']-120)/3 + 120
    else:
        tf_latest = row['arriving_time_latest']
        
    if row['arriving_time_previous'] > 120:
        tf_previous = (row['arriving_time_previous']-120)/3 + 120
    else:
        tf_previous = row['arriving_time_previous']
    return pd.Series({'tf_latest': tf_latest, 'tf_previous': tf_previous})

In [25]:
interpolation_full.loc[s_grids_lon_index:, ['tf_latest', 'tf_previous']] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(hour2timeframe, axis=1))

In [26]:
interpolation_full

,longitude,latitude,cog,timestamp,arriving_time_latest,arriving_time_previous,tf_latest,tf_previous
0,156.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,157.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,158.500007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,159.750007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,161.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5,162.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6,163.500007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,164.750007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,166.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN
9,167.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [27]:
def get_from_minio(filename, bucket=cfg.MINIO_BUCKET_NAME):
    with tempfile.NamedTemporaryFile() as tf:
        minio.fget_object(bucket, filename, tf.name)
        return np.load(tf.name)


if cfg.USE_MINIO:
    minio = Minio(
        endpoint=cfg.MINIO_HOST,
        access_key=cfg.MINIO_ACCESS_KEY,
        secret_key=cfg.MINIO_SECRET_KEY,
        secure=False
    )

    wf_latest = get_from_minio('gfs_NP_'+forecast_name[0]+'.npy')
    print('gfs_NP_'+forecast_name[0]+'.npy')
    wf_previous = get_from_minio('gfs_NP_'+forecast_name[1]+'.npy')
    print('gfs_NP_'+forecast_name[1]+'.npy')

else: 
    wf_latest = np.load('gfs_NP_'+forecast_name[0]+'.npy')
    print('gfs_NP_'+forecast_name[0]+'.npy')
    wf_previous = np.load('gfs_NP_'+forecast_name[1]+'.npy')
    print('gfs_NP_'+forecast_name[1]+'.npy')

gfs_NP_2022101412.npy
gfs_NP_2022101406.npy


In [28]:
#flip for the 1st dimension (latitude) because interpolation requires first dimension in accending order
# slice and drop the last longitude as weather attributes are not needed for the destination
wf_latest = np.flip(wf_latest, axis = 0)[:,:-1,:,:]
wf_previous = np.flip(wf_previous, axis = 0)[:,:-1,:,:]

In [29]:
def interp_row(row, wf, tsn):
    
    '''
    The function aims to retrieve and interpolate the weather conditions at the given position according 
    to its spatial coordinates and arriving time (hour)
    input:
    wf is the weather forecast array, wf_latest or wf_previous
    tsn = 'tf_latest' or 'tf_previous' according to wf
    output: weather conditions for wind, wave and swell (8)
    '''
    # Define the interpolation points as a tuple of arrays
    interp_points = (row['latitude'], row['longitude'], row[tsn], range(wf.shape[3]))

    # Interpolate the weather variable values at the interpolation points
    var_values = interpn((np.flip(lat_list), long_list, range(wf.shape[2]), range(wf.shape[3])),
                     wf, interp_points, method='linear')
    return var_values

In [30]:
# interpolation of weather conditions forecasted in the latest weather forecast and the previous weather forecast
interpolation_full.loc[s_grids_lon_index:,'latest_weather'] = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: 
                                                                                                                 interp_row(row, wf_latest, 'tf_latest'), axis = 1)
interpolation_full.loc[s_grids_lon_index:,'previous_weather'] = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: 
                                                                                                                   interp_row(row, wf_previous, 'tf_previous'), axis = 1)

In [31]:
interpolation_full

,longitude,latitude,cog,timestamp,arriving_time_latest,arriving_time_previous,tf_latest,tf_previous,latest_weather,previous_weather
0,156.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,157.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,158.500007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,159.750007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,161.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5,162.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
6,163.500007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
7,164.750007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8,166.000007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9,167.250007,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Define a function that converts a 1D numpy array of weather variables to the desired format
def convert_weather_array(weather_array, cog):
    # Make a copy of the input array of weather conditions
    converted_array = np.copy(weather_array)

    # Define the indices of the directions of wind, wave and swell
    direction_indices = [1, 4, 7]

    # Subtract the cog from the direction and take the absolute value and convert if larger than 180 degree
    for idx in direction_indices:
        diff = abs(weather_array[idx] - cog)
        if diff <= 180:
            converted_array[idx] = diff
        else:
            converted_array[idx] = 360 - diff

    return converted_array


# Apply the 'convert_weather_array' function to each row of the DataFrame
converted_weather = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: convert_weather_array(
    row['latest_weather'], row['cog']) - convert_weather_array(row['previous_weather'], row['cog']), axis=1)

# Concatenate the converted weather arrays into a single flattened array
flattened_weather = np.concatenate(converted_weather.to_numpy())

# Zero-pad the flattened array if necessary
if len(flattened_weather) < 59 * 8:
    padding = np.zeros((59 * 8 - len(flattened_weather),))
    flattened_weather = np.concatenate((padding, flattened_weather))


print('Final weather array shape:\n', flattened_weather.shape)



Final weather array shape:
 (472,)


In [33]:
flattened_weather

array([  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0. 

### inference

In [34]:
# architecture:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(472, 640) 
        self.layer_2 = nn.Linear(640, 120)
        self.layer_3 = nn.Linear(120, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(640)
        self.batchnorm2 = nn.BatchNorm1d(120)
        self.batchnorm3 = nn.BatchNorm1d(64)
    def init_weights(self):
        torch.nn.init.kaiming_normal_(self.layer_1.weight)
        torch.nn.init.kaiming_normal_(self.layer_2.weight)
        torch.nn.init.kaiming_normal_(self.layer_3.weight)
        torch.nn.init.kaiming_normal_(self.layer_out.weight)      
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout1(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.dropout2(x)
        x = self.layer_out(x)
        
        return x
model = BinaryClassification()

In [35]:
# load the trained model
trained_model = torch.load('pretrained_wr_v2.pt')['state_dict']
model.load_state_dict(trained_model)

<All keys matched successfully>

In [36]:
# predict
tensor_input = torch.tensor(flattened_weather).float().reshape(1, -1)
model.eval()
probs = torch.sigmoid(model.forward(tensor_input))
preds = (probs >= 0.5).int()
print('Rerouting is recommended!' if preds == 1 else 'Rerouting is not recommended!')

Rerouting is not recommended!
